## Authentication of Twitter API

In [2]:
import tweepy
import pandas as pd
import pickle

# Authentication
consumer_api_key = '8AlKwqtWbAIErzUrTOTxiGbnS'
consumer_api_secret_key = 'GODC2dcpV73Q2gc16b2WIOmvOy8aOypvNhqvdNywP0N1cVjHBo'
access_token = '2510554254-r1gHLRt8dv7BqqQq5K1V9PfNrUe3XmFlNAWwitF'
access_token_secret = 'GIfJCfTky0g7MPS4JYSRVBdCcO3HI6FjR0tPz3CgxQyQf'
 
auth = tweepy.OAuthHandler(consumer_api_key, consumer_api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Read the mobileCarrier.pkl, which contains historical data

In [23]:
with open ('mobileCarriers.pkl','rb') as pkl_file:
    df = pickle.load(pkl_file)

df.shape

(213859, 4)

In [22]:
df.head()

,screen_name,date_time,location,text
0,leila_fong,2019-10-23 23:57:59,"Illinois, USA","@ATTHelp I’m blocking, but then they change th..."
1,SnackEvangelist,2019-10-23 23:57:44,,@ATTHelp Just logged into my account and it’s ...
2,DeathboxTy,2019-10-23 23:54:34,,@ATTHelp can’t get my Visual Voicemail to work...
3,BLoLovesYou,2019-10-23 23:54:05,"San Antonio, TX",@ATTHelp You were charging me $150 for an outd...
4,ATTHelp,2019-10-23 23:53:02,,"@leila_fong Hey, Leila, we're here to help. Cl..."


## Task: Fetch new data on a certain date

In [14]:
def getTweets(searchString, df):
    for tweet in tweepy.Cursor(api.search, q=searchString, lang='en', since='2019-10-23', until='2019-10-24').items():
        df = df.append(pd.Series([tweet.user.screen_name, tweet.created_at, tweet.user.location, tweet.text], index=df.columns), ignore_index=True)
    return df

In [15]:
# dataframe in one day batch
df_tmp = pd.DataFrame(columns=['screen_name', 'date_time', 'location', 'text'])

In [16]:
df_tmp = getTweets('VerizonSupport', df_tmp)
df_tmp = getTweets('verizon', df_tmp)
df_tmp = getTweets('VZWSupport', df_tmp)
df_tmp = getTweets('ATT', df_tmp)
df_tmp = getTweets('ATTHelp', df_tmp)
df_tmp = getTweets('TMobile', df_tmp)
df_tmp = getTweets('TMobileHelp', df_tmp)

Rate limit reached. Sleeping for: 242


In [17]:
df_tmp.head(), df_tmp.shape

(       screen_name           date_time         location  \
 0       leila_fong 2019-10-23 23:57:59    Illinois, USA   
 1  SnackEvangelist 2019-10-23 23:57:44                    
 2       DeathboxTy 2019-10-23 23:54:34                    
 3      BLoLovesYou 2019-10-23 23:54:05  San Antonio, TX   
 4          ATTHelp 2019-10-23 23:53:02                    
 
                                                 text  
 0  @ATTHelp I’m blocking, but then they change th...  
 1  @ATTHelp Just logged into my account and it’s ...  
 2  @ATTHelp can’t get my Visual Voicemail to work...  
 3  @ATTHelp You were charging me $150 for an outd...  
 4  @leila_fong Hey, Leila, we're here to help. Cl...  , (4497, 4))

## Concat the new data to previous dataframe

In [18]:
df = pd.concat([df_tmp,df], ignore_index=True)

In [19]:
df.shape

(213859, 4)

## Store all data into the pickle

In [20]:
with open('mobileCarriers.pkl', 'wb') as pkl_file:
    pickle.dump(df, pkl_file) 